# Предобработка команд PowerShell с помощью утилиты Revoke-Obfuscation для обучения моделей/классификации

- Исходный код утилиты доступен на GitHub по [ссылке](https://github.com/danielbohannon/Revoke-Obfuscation)
- Некоторые необходимые данные можно скачать из облака по [ссылке](https://drive.google.com/drive/folders/1TN4UM1v2XJUxkAhCQxKBge_TsyY9FgCs?usp=drive_link)

## Инструкция по настройке:

- Необходимо установить `PowerShell` (7.x версии и выше)
- Необходимо установить `dotnet` (6.0 версии)
- Необходимо установить `PSFeatureExtractorLibrary.zip` и `pspython.runtimeconfig.json` из облака
- Необходимо скачать файл `drop_features.txt` из облака, содержащий признаки, которые необходимо удалить

*Если версия `dotnet` отличается, то необходимо:*
1. пересобрать DLL библиотеку, содержащую методы Revoke-Obfuscation
2. поменять конфигурационные данные в файле `pspython.runtimeconfig.json`

*Исходные используемые классы Revoke-Obfuscation доступны в облаке в архиве `Revoke-Obfuscation_source-code.zip`*

## Инструкция по использованию:

- Для обучения моделей ожидается CSV файл, построчно содержащий команду PowerShell и результат её классификации (header отсутствует, то есть сразу идут строки в формате '<команда PowerShell>,<обфусцированность (0 или 1)>')
- Для классификации ожидается TXT файл, построчно содержащий команды PowerShell
- На выходе получается предобработанный набор данных (CSV файл), содержащий 2709 признаков, среди которых для обучения признак `obfuscated` является целевым, я для классификации признак `command` содержит команду PowerShell до предобработки

*При необходимости редактируйте ноутбук под свои условия*

### 1. Настройка и подключение утилиты Revoke-Obfuscation

In [ ]:
import os 
import sys

import pythonnet
import clr_loader
import numpy as np
import pandas as pd

In [ ]:
dotnet_path = r"<путь до dotnet>"
powershell_path = r"<путь до PowerShell>"
config_file_path = r"<путь до pspython.runtimeconfig.json>"
ps_features_extractor_library_path = r"<путь до PSFeatureExtractorLibrary> (net6.0 из архива или перегенерированное решение)"

microsoft_management_infrastructure_dll_path = fr"{powershell_path}\Microsoft.Management.Infrastructure.dll"
system_management_automation_dll_path = fr"{powershell_path}\System.Management.Automation.dll"

In [ ]:
os.environ["DOTNET_ROOT"] = dotnet_path

In [ ]:
coreclr = clr_loader.get_coreclr(runtime_config=config_file_path)
pythonnet.set_runtime(coreclr)
pythonnet.get_runtime_info()

In [ ]:
import clr
import System

clr.AddReference(microsoft_management_infrastructure_dll_path)
clr.AddReference(system_management_automation_dll_path)

# test adding references
# import Microsoft.Management.Infrastructure
# import System.Management.Automation.Language

In [ ]:
sys.path.append(ps_features_extractor_library_path)
clr.AddReference("PSFeatureExtractorLibrary")

# test adding reference
# import PSFeatureExtractorLibrary

### 2. Токенизация

In [ ]:
import PSFeatureExtractorLibrary

METHODS = [
    PSFeatureExtractorLibrary.GroupedArrayElementRangeCounts.AnalyzeAst,
    PSFeatureExtractorLibrary.ArrayElementMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.GroupedAssignmentStatements.AnalyzeAst,
    PSFeatureExtractorLibrary.GroupedAstTypes.AnalyzeAst,
    PSFeatureExtractorLibrary.GroupedBinaryExpressionOperators.AnalyzeAst,
    PSFeatureExtractorLibrary.CmdletMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.CommandParameterNameMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.CommentMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.ConvertExpressionMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.FunctionNameMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.IntegerAndDoubleMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.InvocationOperatorInvokedObjectMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.LineByLineMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.MemberArgumentMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.MemberMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.StringMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.TypeConstraintMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.TypeExpressionMetrics.AnalyzeAst,
    PSFeatureExtractorLibrary.GroupedUnaryExpressionOperators.AnalyzeAst,
    PSFeatureExtractorLibrary.VariableNameMetrics.AnalyzeAst
]

FEATURES_PER_METHOD = [22, 312, 14, 152, 104, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 26, 312]
CUMULATIVE_FEATURES = np.cumsum(FEATURES_PER_METHOD)

In [ ]:
def tokenize_without_features(command: str, methods: list, cumulative_features: list[int]) -> list[float]:
    ast, token, parsing_error = System.Management.Automation.Language.Parser.ParseInput(command)
    
    if parsing_error:
        raise ValueError(parsing_error)
    
    tokenized_values = np.zeros(cumulative_features[-1], dtype="float64")
    
    left_index = 0
    for i in range(len(methods)):
        right_index = cumulative_features[i]
        tokenized_values[left_index:right_index] = list(methods[i](ast).Values)
        left_index = right_index

    return list(tokenized_values)

In [ ]:
def tokenize_with_features(command: str, methods: list, cumulative_features: list[int]) -> dict[str, list]:
    ast, token, parsing_error = System.Management.Automation.Language.Parser.ParseInput(command)
    
    if parsing_error:
        raise ValueError(parsing_error)
        
    tokenized_values = np.zeros(cumulative_features[-1], dtype="float64")
    features = [''] * cumulative_features[-1]
    
    left_index = 0
    for i in range(len(methods)):
        right_index = cumulative_features[i]
        ast_analysis = methods[i](ast)
        tokenized_values[left_index:right_index] = list(ast_analysis.Values)
        features[left_index:right_index] = list(ast_analysis.Keys)
        left_index = right_index

    return {"features": features, "values": list(tokenized_values)}

In [ ]:
def tokenize(commands: list[str]) -> tuple[pd.DataFrame, list[str]]:
    dataframe = pd.DataFrame()
    
    count = 0
    tokenized_commands = []
    for command in commands:
        try:
            if not count:
                tokenized_result = tokenize_with_features(command=command, methods=METHODS, cumulative_features=CUMULATIVE_FEATURES)
                dataframe = pd.concat([dataframe, pd.DataFrame(columns=tokenized_result["features"])], ignore_index=True)
                dataframe.loc[len(dataframe)] = tokenized_result["values"]
            else:
                dataframe.loc[len(dataframe)] = tokenize_without_features(command=command, methods=METHODS, cumulative_features=CUMULATIVE_FEATURES)
            count += 1
            if not count % 500:
                print(f"{count} commands tokenized successfully\n")
            tokenized_commands.append(command)
        except Exception as e:
            print(f"For command {command!r} error occurred while tokenizing: {e}\n")
    return dataframe, tokenized_commands

### 3. Предобработка

In [ ]:
def drop_columns(dataframe: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    return dataframe.drop(labels=columns, axis=1)

In [ ]:
def add_column(dataframe: pd.DataFrame, column_name: str, values: list) -> pd.DataFrame:
    dataframe[column_name] = values
    return dataframe

### 4. Считывание данных из файла

- для получения команд перед классификацией
- для получения признаков, которые нужно исключить

In [ ]:
def read_txt_file(filename: str) -> list[str]:
    with open(file=filename, mode='r') as file:
        return [row for row in file.read().split('\n') if row]

### 5. Считывание исходных данных, токенизация, предобработка и сохранение полученных данных

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def preprocess_for_training(dataframe: pd.DataFrame, features_to_drop_path: str) -> pd.DataFrame:
    if len(dataframe) < 2:
        raise ValueError(f"Input data must contains at least 2 commands")
    commands = dataframe.iloc[:, 0]
    y = dataframe.iloc[:, 1]
    tokenized_dataframe, _ = tokenize(commands=commands)
    features_to_drop = read_txt_file(filename=features_to_drop_path)
    balanced_dataframe = drop_columns(dataframe=tokenized_dataframe, columns=features_to_drop)
    normalized_dataframe = pd.DataFrame(MinMaxScaler().fit_transform(balanced_dataframe), columns=balanced_dataframe.columns)
    return add_column(dataframe=normalized_dataframe, column_name="obfuscated", values=y)

In [ ]:
def preprocess_for_classification(commands: list[str], features_to_drop_path: str) -> pd.DataFrame:
    if len(commands) < 2:
        raise ValueError(f"Input data must contains at least 2 commands")
    tokenized_dataframe, tokenized_commands = tokenize(commands=commands)
    features_to_drop = read_txt_file(filename=features_to_drop_path)
    balanced_dataframe = drop_columns(dataframe=tokenized_dataframe, columns=features_to_drop)
    normalized_dataframe = pd.DataFrame(MinMaxScaler().fit_transform(balanced_dataframe), columns=balanced_dataframe.columns)
    return add_column(dataframe=normalized_dataframe, column_name="command", values=tokenized_commands)

In [ ]:
features_to_drop_path = r"<путь до файла drop_features.txt>"

In [ ]:
# Предобработка для обучения
source_filename = r"<путь до исходного *.csv файла с командами PowerShell и значениями их обфусцированности для обучения моделей>"
target_filename = r"<путь до *.csv файла, куда будет сохранен результат предобработки>"

source_dataframe = pd.read_csv(filepath_or_buffer=source_filename, header=None)
preprocessed_dataframe = preprocess_for_training(dataframe=source_dataframe, features_to_drop_path=features_to_drop_path)
preprocessed_dataframe.to_csv(target_filename, index=False)

In [ ]:
# Предобработка для классификации
source_filename = r"<путь до исходного *.txt файла с командами PowerShell для классификации>"
target_filename = r"<путь до *.csv файла, куда будет сохранен результат предобработки>"

commands = read_txt_file(filename=source_filename)
preprocessed_dataframe = preprocess_for_classification(commands=commands, features_to_drop_path=features_to_drop_path)
preprocessed_dataframe.to_csv(target_filename, index=False)